In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import nltk.corpus
from sklearn.feature_extraction.text import TfidfVectorizer
from TurkishStemmer import TurkishStemmer
from sklearn import cluster
from sklearn import feature_extraction
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from itertools import chain
from TurkishStemmer import TurkishStemmer
import matplotlib.pyplot as plt

In [3]:
df = pd.read_pickle('./clustered_test_998_group_4.pkl')
df = df.drop(df.columns[[1,2,4,5,3]], axis=1)
pd.set_option('max_colwidth', None)
df.head(1)

,Content
14053,"Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde\nIMF Euro Bölgesi büyüme öngörüsünü altı yılın en düşük düzeyine çekti\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü.\nEuro Bölgesi için 2020 ve 2021 yılına ilişkin büyüme tahminlerini yüzde 1.5'ten yüzde 1.4'e düşüren IMF, düşüşün nedeni olarak Almanya ve İtalya ekonomilerinde görülen zayıflamayı gösterdi.IMF, 19 ülkeden oluşan Euro Bölgesi'nin bu yıl için büyüme öngörüsünü, daha önceki yüzde 1.3 düzeyinden yüzde 1.2'ye revize etti. Euro Bölgesi'nin 2018 büyümesi yüzde 1.9 düzeyindeydi.\nAlmanya ve İtalya için önceki büyüme tahminlerini revize eden IMF, yavaşlamanın çoğunlukla euro bölgesinin en büyük ekonomisi Almanya'daki zayıf büyüme ve üçüncü büyük ekonomisi İtalya'daki durgunluktan kaynaklandığını ifade etti.\nIMF Almanya ekonomisindeki büyüme beklentisini Nisan ayı tahmini olan yüzde 0.8'den yüzde 0.5'e ve İtalyan ekonomisinin de duraklayacağını öngörerek büyüme beklentisini yüzde 0.9'dan yüzde 0.5'e düşürdü.\nETİKETLER :\n \n IMF\n euro bölgesi"


In [4]:
myRegex = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
stop_word_list = nltk.corpus.stopwords.words('turkish')
def clean_text(text):
    text = text.lower()
    text = re.sub('\[*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(myRegex), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\t', '', text)
    text = re.sub('\n', '', text)
    word_tokens = nltk.word_tokenize(text)
    filtered_sentences = [w for w in word_tokens if not w in stop_word_list]

    text = ' '.join(filtered_sentences)
    return text
    
round1 = lambda x: clean_text(x)

df = pd.DataFrame(df['Content'].apply(round1))

In [5]:
df['ready_data'] = df['Content'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df.head(1)


,Content,ready_data
14053,euro bölgesi büyüme öngörüsü altı yılın düşüğündeimf euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu imf bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt içi hasılasının son altı yılın düşük büyüme hızına gerileyerek yılında yüzde . düşüşle yüzde . düzeyinde gerçekleşeceğini öngördü.euro bölgesi yılına ilişkin büyüme tahminlerini yüzde . ten yüzde . e düşüren imf düşüşün nedeni olarak almanya i̇talya ekonomilerinde görülen zayıflamayı gösterdi.imf ülkeden oluşan euro bölgesi nin yıl büyüme öngörüsünü önceki yüzde . düzeyinden yüzde . ye revize etti . euro bölgesi nin büyümesi yüzde . düzeyindeydi.almanya i̇talya önceki büyüme tahminlerini revize eden imf yavaşlamanın çoğunlukla euro bölgesinin büyük ekonomisi almanya daki zayıf büyüme üçüncü büyük ekonomisi i̇talya daki durgunluktan kaynaklandığını ifade etti.imf almanya ekonomisindeki büyüme beklentisini nisan ayı tahmini olan yüzde . den yüzde . e i̇talyan ekonomisinin duraklayacağını öngörerek büyüme beklentisini yüzde . dan yüzde . e düşürdü.eti̇ketler imf euro bölgesi,euro bölgesi büyüme öngörüsü altı yılın düşüğündeimf euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü.euro bölgesi yılına ilişkin büyüme tahminlerini yüzde yüzde düşüren düşüşün nedeni olarak almanya i̇talya ekonomilerinde görülen zayıflamayı gösterdi.imf ülkeden oluşan euro bölgesi büyüme öngörüsünü önceki yüzde düzeyinden yüzde revize etti euro bölgesi büyümesi yüzde düzeyindeydi.almanya i̇talya önceki büyüme tahminlerini revize eden yavaşlamanın çoğunlukla euro bölgesinin büyük ekonomisi almanya daki zayıf büyüme üçüncü büyük ekonomisi i̇talya daki durgunluktan kaynaklandığını ifade etti.imf almanya ekonomisindeki büyüme beklentisini nisan tahmini olan yüzde yüzde i̇talyan ekonomisinin duraklayacağını öngörerek büyüme beklentisini yüzde yüzde düşürdü.eti̇ketler euro bölgesi


In [6]:
tokenized_doc = df['ready_data'].apply(lambda x: x.split('.'))
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_word_list])
tokenized_doc = tokenized_doc.tolist()

In [7]:
detokenized_doc = []
for i in range(len(tokenized_doc)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    #if i in tokenized_doc:
        
len(detokenized_doc)

998

In [8]:
vectorizer = TfidfVectorizer(stop_words=stop_word_list,max_features=200000,
                             max_df = 0.8, use_idf = True, ngram_range=(1,3))
X = vectorizer.fit_transform(detokenized_doc)
print(X.shape) 
terms = vectorizer.get_feature_names()

(998, 54673)


In [10]:
from sklearn import preprocessing
XXX = preprocessing.normalize(X)
num_clusters = 4
km = KMeans(n_clusters=num_clusters).fit(XXX)
clusters = km.labels_.tolist()

In [11]:
#burdasi tahmin etmek icin. bunlari fonksiyona cevir
df_test = pd.read_pickle('./clustered_test_40300_all_group.pkl')
"""df_test = df_test.drop(df_test.columns[[1,3,4,5,6,7,8]], axis=1)
test_data = df_test.values.tolist()
test_data[0]"""

df_test = pd.DataFrame(df_test['Content'].apply(round1))
df_test['ready_data'] = df_test['Content'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

test_tokenized_doc = df_test['ready_data'].apply(lambda x: x.split('.'))
test_tokenized_doc = test_tokenized_doc.apply(lambda x: [item for item in x if item not in stop_word_list])
test_tokenized_doc = test_tokenized_doc.tolist()

test_detokenized_doc = []
for i in range(len(test_tokenized_doc)):
    t = ' '.join(test_tokenized_doc[i])
    test_detokenized_doc.append(t)


test_vectorizer = TfidfVectorizer(stop_words=stop_word_list, max_features=200000,
                             max_df = 0.8, use_idf = True, ngram_range=(1,3))

In [12]:
XX = test_vectorizer.fit_transform(test_detokenized_doc)
print(XX.shape) # check shape of the document-term matrix

(998, 54673)


In [13]:
test_data_clusters = []
for i in range(998):
    test_data_clusters.append(km.predict(XX[i]))

In [14]:
#benim tahmin ettiklerim
df_test['Cluster'] = test_data_clusters
df_test[['ready_data','Cluster']].head(-20)
#df_test.to_csv(r'/home/ak/Desktop/clustered_test_datasets/pandas.csv', header=None, index=None, sep=' ', mode='a')

,ready_data,Cluster
14053,euro bölgesi büyüme öngörüsü altı yılın düşüğündeimf euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü.euro bölgesi yılına ilişkin büyüme tahminlerini yüzde yüzde düşüren düşüşün nedeni olarak almanya i̇talya ekonomilerinde görülen zayıflamayı gösterdi.imf ülkeden oluşan euro bölgesi büyüme öngörüsünü önceki yüzde düzeyinden yüzde revize etti euro bölgesi büyümesi yüzde düzeyindeydi.almanya i̇talya önceki büyüme tahminlerini revize eden yavaşlamanın çoğunlukla euro bölgesinin büyük ekonomisi almanya daki zayıf büyüme üçüncü büyük ekonomisi i̇talya daki durgunluktan kaynaklandığını ifade etti.imf almanya ekonomisindeki büyüme beklentisini nisan tahmini olan yüzde yüzde i̇talyan ekonomisinin duraklayacağını öngörerek büyüme beklentisini yüzde yüzde düşürdü.eti̇ketler euro bölgesi,[3]
14148,euro bölgesi büyüme öngörüsü altı yılın düşüğündeimf euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü euro bölgesi yılına ilişkin büyüme tahminlerini yüzde yüzde düşüren düşüşün nedeni olarak almanya i̇talya ekonomilerinde görülen zayıflamayı gösterdi ülkeden oluşan euro bölgesi büyüme öngörüsünü önceki yüzde düzeyinden yüzde revize etti euro bölgesi büyümesi yüzde düzeyindeydi almanya i̇talya önceki büyüme tahminlerini revize eden yavaşlamanın çoğunlukla euro bölgesinin büyük ekonomisi almanya daki zayıf büyüme üçüncü büyük ekonomisi i̇talya daki durgunluktan kaynaklandığını ifade etti almanya ekonomisindeki büyüme beklentisini nisan tahmini olan yüzde yüzde i̇talyan ekonomisinin duraklayacağını öngörerek büyüme beklentisini yüzde yüzde düşürdü sayfada alan bilgiler tavsiye niteliği taşımayıp yatırım danışmanlığı kapsamında değildir yatırımcı profilinize uymayabilir,[3]
13559,euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü.uluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü euro bölgesi yılına ilişkin büyüme tahminlerini yüzde yüzde düşüren düşüşün nedeni olarak almanya i̇talya ekonomilerinde görülen zayıflamayı gösterdi.imf ülkeden oluşan euro bölgesi büyüme öngörüsünü önceki yüzde düzeyinden yüzde revize etti euro bölgesi büyümesi yüzde düzeyindeydi.almanya i̇talya önceki büyüme tahminlerini revize eden yavaşlamanın çoğunlukla euro bölgesinin büyük ekonomisi almanya daki zayıf büyüme üçüncü büyük ekonomisi i̇talya daki durgunluktan kaynaklandığını ifade etti.imf almanya ekonomisindeki büyüme beklentisini nisan tahmini olan yüzde yüzde i̇talyan ekonomisinin duraklayacağını öngörerek büyüme beklentisini yüzde yüzde düşürdü kaynak,[3]
13566,euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektihaber euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çekti uluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çekti uluslararası para fonu bölgesel ekonomik görünüm raporunda euro bölgesi gayrisafi yurt hasılasının altı yılın düşük büyüme hızına gerileyerek yılında yüzde düşüşle yüzde düzeyinde gerçekleşeceğini öngördü,[3]
13894,euro bölgesi büyüme öngörüsünü altı yılın düşük düzeyine çektiuluslararası para fonu 

In [16]:
#Test verisetinden bana gelen veriler.
#Test verileri ile çeşitli veriler arasında karşılaştırmalar yapıldı.
#Bu karşılaştırmalardan birkaçı aşağıdaki gibidir.
df_test2 = pd.read_pickle('./clustered_test_40300_all_group.pkl')
#df_test2 = df_test2.drop(df.columns[[0,3,4,5,6,7,8,9]], axis=1)
df_test2.head(-20)

,ClusterCalcTime,ClusterUuID,Content,DocumentTypeID,ID,OldClusterUuID,SimilarityDocID,SimilarityScore,Title,UpdateDate
14053,1161,06f05f39-0ff9-4e60-ae16-d7401ffec0d3,"Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde\nIMF Euro Bölgesi büyüme öngörüsünü altı yılın en düşük düzeyine çekti\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü.\nEuro Bölgesi için 2020 ve 2021 yılına ilişkin büyüme tahminlerini yüzde 1.5'ten yüzde 1.4'e düşüren IMF, düşüşün nedeni olarak Almanya ve İtalya ekonomilerinde görülen zayıflamayı gösterdi.IMF, 19 ülkeden oluşan Euro Bölgesi'nin bu yıl için büyüme öngörüsünü, daha önceki yüzde 1.3 düzeyinden yüzde 1.2'ye revize etti. Euro Bölgesi'nin 2018 büyümesi yüzde 1.9 düzeyindeydi.\nAlmanya ve İtalya için önceki büyüme tahminlerini revize eden IMF, yavaşlamanın çoğunlukla euro bölgesinin en büyük ekonomisi Almanya'daki zayıf büyüme ve üçüncü büyük ekonomisi İtalya'daki durgunluktan kaynaklandığını ifade etti.\nIMF Almanya ekonomisindeki büyüme beklentisini Nisan ayı tahmini olan yüzde 0.8'den yüzde 0.5'e ve İtalyan ekonomisinin de duraklayacağını öngörerek büyüme beklentisini yüzde 0.9'dan yüzde 0.5'e düşürdü.\nETİKETLER :\n \n IMF\n euro bölgesi",8,1482482922,5e7118e0-4720-4437-b417-c497ef237e5d,0,0.00,Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde,2020-01-12T12:27:02Z
14148,1298,06f05f39-0ff9-4e60-ae16-d7401ffec0d3,"Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde\nIMF Euro Bölgesi büyüme öngörüsünü altı yılın en düşük düzeyine çekti\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü. Euro Bölgesi için 2020 ve 2021 yılına ilişkin büyüme tahminlerini yüzde 1.5'ten yüzde 1.4'e düşüren IMF, düşüşün nedeni olarak Almanya ve İtalya ekonomilerinde görülen zayıflamayı gösterdi. IMF, 19 ülkeden oluşan Euro Bölgesi'nin bu yıl için büyüme öngörüsünü, daha önceki yüzde 1.3 düzeyinden yüzde 1.2'ye revize etti. Euro Bölgesi'nin 2018 büyümesi yüzde 1.9 düzeyindeydi. Almanya ve İtalya için önceki büyüme tahminlerini revize eden IMF, yavaşlamanın çoğunlukla euro bölgesinin en büyük ekonomisi Almanya'daki zayıf büyüme ve üçüncü büyük ekonomisi İtalya'daki durgunluktan kaynaklandığını ifade etti. IMF Almanya ekonomisindeki büyüme beklentisini Nisan ayı tahmini olan yüzde 0.8'den yüzde 0.5'e ve İtalyan ekonomisinin de duraklayacağını öngörerek büyüme beklentisini yüzde 0.9'dan yüzde 0.5'e düşürdü. \n \n\t\t\t\tSayfada yer alan bilgiler tavsiye niteliği taşımayıp yatırım danışmanlığı kapsamında değildir. Yatırımcı profilinize uymayabilir.\n",8,1482483155,5e7118e0-4720-4437-b417-c497ef237e5d,1482482922,99.35,Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde,2020-01-12T12:32:18Z
13559,1000,06f05f39-0ff9-4e60-ae16-d7401ffec0d3,"IMF Euro Bölgesi büyüme öngörüsünü altı yılın en düşük düzeyine çekti\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü.\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü. Euro Bölgesi için 2020 ve 2021 yılına ilişkin büyüme tahminlerini yüzde 1.5'ten yüzde 1.4'e düşüren IMF, düşüşün nedeni olarak Almanya ve İtalya ekonomilerinde görülen zayıflamayı gösterdi.\nIMF, 19 ülkeden oluşan Euro Bölgesi'nin bu yıl için büyüme öngörüsünü, daha önceki yüzde 1.3 düzeyinden yüzde 1.2'ye revize etti. Euro Bölgesi'nin 2018 büyümesi yüzde 1.9 düzeyindeydi.Almanya

In [17]:
from sklearn.utils.extmath import randomized_svd
U, Sigma, VT = randomized_svd(XXX, n_components=4, n_iter=100,
                              random_state=122)

for i, comp in enumerate(VT):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=False)[:10]
    print("Haber "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
    print(" ")

Haber 0: 
adalet kalkınma
adalet kalkınma partisi
alana gidiyoruz
alana gidiyoruz emeklilerle
aldıkları emekli
aldıkları emekli ikramiyesi
almalarını
almalarını sağlamak
almalarını sağlamak komisyonda
altını çizen
 
Haber 1: 
albayrak
sahibi
enflasyon
kısmı
olacak
enflasyon olacak
enflasyon olacak başlangıç
olacak başlangıç
yılda
birileri
 
Haber 2: 
yüzde
endeksi
endeksi yüzde
borsa
bist endeksi
yükselişle
güne
güne yükselişle
yükselişle başladı
bankacılık
 
Haber 3: 
yatırım
com
http
burada alan
yorum
mücadele
alan
foreks
burada
şekilde
 
